# CHAPTER 1 - Approximation and Fitting Problems



---
---

**Author:** Dr Giordano Scarciotti (g.scarciotti@imperial.ac.uk) - Imperial College London 

**Module:** ELEC70066 - Advanced Optimisation

**Version:** 1.1.2 - 19/01/2023

---
---

The material of this chapter is adapted from the book *Boyd & Vandenberghe "Convex Optimization"* which from now on we refer to as $[1]$. The code is adapted from the documentation of CVXPY at https://www.cvxpy.org

This chapter covers the class of applied convex optimisation problems in the fields of approximation and fitting. The chapter consists of five sections: 

*   In Section 1.1 we introduce the basic norm approximation problem. We look at several applications and explore what impact the selection of the penalty function has on the solution of the problem.
*   In Section 1.2 we look at least-norm problems.
*   In Section 1.3 we look at regularised approximation as a way of combining the first two classes of problems and as a first example of how to deal with multi-objective optimisation.
*   In Section 1.4 we look at robust approximation, a somewhat more advanced class of applications.
*   Section 1.5 is dedicated to introduce CVXPY.



**Nota bene:** The purpose of this chapter is to expose you immediately to important classes of applied optimisation problems to motivate the course. You should not expect to be able to grasp the mathematical details at this stage. Later in the course you will learn all the required knowledge to fully understand the theory of this chapter.

# 1.1 Basic Norm Approximation Problem

In [ ]:
# Ignore this code; it is just used to generate the youtube video below
from IPython.display import HTML
HTML('<iframe width="850" height="480" src="https://www.youtube.com/embed/H-P8kTSu8bo"></iframe>')

Let $x_1, \dots, x_n$ be $n$ unknown design variables that we want to determine. Consider $y=Ax$ as a vector containing $m$ results which are produced as a linear combinations of the design variables $x$. Let $b$ be a vector of targets or desired results. The problem that we want to solve is to choose the vector of design variables $x$ to achieve, as
closely as possible, the desired results, i.e., $Ax \approx b$. We can interpret the residual vector $r= Ax-b $ as the deviation between the actual results (i.e., $Ax$) and the desired
or target results (i.e., $b$). To measure the quality of a design we can choose a norm to weigh the deviation between the actual results and the desired results. Mathematically the problem that we want to solve is

$$\displaystyle\min_x || Ax-b ||\tag{1}$$

where $A\in \mathbb{R}^{m\times n}$, $b \in \mathbb{R}^m$, $x\in \mathbb{R}^n$ and $|| \cdot ||$ is a norm. Problem $(1)$ is called norm approximation problem and it is a simple example of *unconstrained convex optimisation problem*. Problem $(1)$ is  solvable, i.e., there is always at least an optimal solution. There are trivial, uninteresting, solutions when $b\in\mathcal{R}(A)$, where $\mathcal{R}(A)$ indicates the range of $A$, or when $m = n$. In the first case the optimal value $p^*$ is $0$ and in the second case the optimal solution $x^*$ is $A^{-1}b$. The problem is then interesting only when $b\not \in\mathcal{R}(A)$ and $m>n$, i.e., we do not have enough design variables to obtain zero residual error. We can assume without loss of generality that the rank of $A$ is full (if not, just remove some redundant lines).

## 1.1.1 One formulation, many problems

Problem (1) has been introduced as a problem of **optimal design**. In reality this problem arises in a multitude of different applications. 

**Regression problem:** By expressing $Ax$ as $Ax = x_1a_1 + \dots + x_na_n$, where $a_1, \dots, a_n \in \mathbb{R}^m$ are the columns of $A$, we see that the goal of the norm approximation problem is to fit or approximate the vector $b$ by a linear combination of the columns of $A$. In this case the vectors $a_1, \dots, a_n $ are called *regressors* and the optimal vector $x_1^*a_1 + \dots + x_n^*a_n$, where $x^*$ is the optimal solution is called *regression* of $b$.

**Estimation problem:** Given a device, we want to estimate certain physical parameters $x\in \mathbb{R}^n$ of the device. To this end, we take some measurements $\tilde y$ that are a linear combination of the $x$ plus some additive noise $v$ (aka measurement error), i.e.,

$$
\tilde  y=Ax + v.
$$

The error is unknown but presumed to be small in a certain norm $|| \cdot ||$. The estimation problem consists in making a guess of $x$ given $y$. This is again problem (1) with $b=\tilde  y$ and $r=v$ ($=\tilde y-Ax=b-Ax$).

**Projection problem:** Consider the subspace $\mathcal{A} = \mathcal{R}(A)$, and a point $b\in\mathbb{R}^m$. A projection of the point $b$ onto the subspace $\mathcal{A}$, in the norm $|| \cdot ||$, is any point in $\mathcal{A}$ that is closest to $b$. By parametrizing an arbitrary element of $\mathcal{R}(A)$ as $u = Ax$,  we see that solving the norm approximation problem (1) is equivalent to computing a projection of $b$ onto $\mathcal{A}$.



## 1.1.2 The usual norms

We consider now three commonly used norms: the euclidean norm, the supremum norm and the $\ell_1$-norm.


**Least-squares approximation:** The most common norm approximation problem involves the Euclidean or $\ell_2$-norm. If in particular we consider the squared Euclidean norm, then problem (1) is called *least-square approximation* problem

$$
\displaystyle\min_x || Ax-b ||_2^2 = r_1^2 + \dots + r_m^2
$$

because the objective is to minimize the sum of the squares of the residuals. This is a problem for which we can compute the analytical solution: $x = (A^\top A)^{-1} A^{\top} b$. Note that it is very rare that an interesting problem has an [analytical solution](https://math.stackexchange.com/a/935458).

***Exercise 1.1:*** Prove that $x = (A^\top A)^{-1} A^{\top} b$

***EDIT THE FILE TO ADD YOUR PROOF HERE***

***Example 1.1:*** In the following snippet of code, we solve a least-square approximation problem in CVXPY. It is OK not being able to understand this code at this point. 

In [ ]:
# <<<=== CLICK ON THE PLAY BUTTON TO RUN THE CODE
# This code is taken from https://www.cvxpy.org/examples/basic/least_squares.html
# Import packages.
import cvxpy as cp # Import CVXPY library
import numpy as np # Import NumPy library

# Generate data.
m = 20
n = 18
np.random.seed(1) # Initialise the seed of the random generator
A = np.random.randn(m, n) # Generate an m by n random matrix
b = np.random.randn(m) # Generate an m by 1 random vector

# Define and solve the CVXPY problem.
x = cp.Variable(n) # Tell Python that x is a cvxpy variable. Now x is special
cost = cp.sum_squares(A @ x - b) # Define the cost function
prob = cp.Problem(cp.Minimize(cost)) # Define the problem: min cost
prob.solve() # Solve the problem

# Print result.
print("\nThe optimal value is", prob.value)
print("The optimal x is")
print(x.value)
print("The norm of the residual is ", cp.norm(A @ x - b, p=2).value)


The optimal value is 0.5354907859470996
The optimal x is
[-0.55643692  0.72378359 -0.74305514  0.06134283  0.02435594  0.3667285
  1.12633113  0.25633759 -0.45381919  0.81670344 -0.75221494 -0.40353227
  0.99761093 -0.31361104  0.06810759  0.5395402  -0.70633702 -0.8296067 ]
The norm of the residual is  0.731772359376261


**Supremum norm:** When in Problem (1) we use the supremum norm, i.e. $\ell_\infty$-norm,

$$
\displaystyle\min_x || Ax-b ||_\infty = \max \{|r_1|, \dots, |r_m|\} \tag{2}
$$

we have the so-called *Chebyshev or minmax approximation problem* since we want to minimize the maximum absolute value of the residuals. Note that the Chebyshev approximation problem can be cast as a linear programme (LP)

$$
\begin{array}{ll}
\displaystyle \min_{t,x} &t\\ 
\text{s.t. } &-t \mathbf{1} \preccurlyeq Ax-b \preccurlyeq t\mathbf{1}
\end{array} \tag{3}
$$

with variables $x \in \mathbb{R}^n$ and $t \in\mathbb{R}$ and where $\preccurlyeq$ indicates the component-wise inequality and $\mathbf{1}$ indicates a vector with all elements equal to $1$.












***Exercise 1.2:*** Show that (2) can be rewritten as (3).

***EDIT THE FILE TO ADD YOUR WORK HERE***

***Example 1.2:*** In the following snippet of code, we solve a minmax approximation problem in CVXPY. Note that CVXPY does not require that you rewrite (2) into (3). It will do it automatically internally. As an exercise you could try to write and solve the LP formulation in CVXPY, but it is OK not being able to do this now. Come back here later in the module once you know more about CVXPY.

In [ ]:
# This snippet requires that you run the code in Example 1.1 first.
# Define and solve the CVXPY problem.
x_inf = cp.Variable(n)
cost_inf = cp.max(cp.abs(A @ x_inf - b))
prob_inf = cp.Problem(cp.Minimize(cost_inf))
prob_inf.solve()

# Print result.
print("\nThe optimal value is", prob_inf.value)
print("The optimal x is")
print(x_inf.value)


The optimal value is 0.21933447223683555
The optimal x is
[-0.49624593  0.73278356 -0.61738623  0.04104026  0.02132238  0.36893673
  1.19883414  0.33983393 -0.47810499  0.81497387 -0.75441444 -0.45694551
  1.13590387 -0.45135143  0.08894124  0.60449616 -0.75526953 -0.88384739]


**$\ell_1$-norm:** When in problem (1) we use the $\ell_1$-norm,

$$
\displaystyle\min_x || Ax-b ||_1 = |r_1| + \dots + |r_m| \tag{4}
$$

we have the so-called *robust estimation problem*. Also this problem that at first looks difficult can be recast as a linear programme (LP)

$$
\begin{array}{ll}
\displaystyle \min_{t,x} &\mathbf{1}^{\top}t\\ 
\text{s.t. } &-t  \preccurlyeq Ax-b \preccurlyeq t
\end{array} \tag{5}
$$

with variables $x \in \mathbb{R}^n$ and $t \in\mathbb{R}^m$.

***Exercise 1.3:*** Show that (4) can be rewritten as (5).

***EDIT THE FILE TO ADD YOUR PROOF HERE***

***Exercise 1.5:*** Write a code snippet to solve the robust estimation problem.

In [ ]:
# Write your code here

## 1.1.3 From norms to general penalty functions

In the $\ell_p$-norm problems above, the objective is 
$$
(|r_1|+\dots + |r_m|^p)^{\frac{1}{p}}.
$$
We now consider the generalisation
$$
\begin{array}{ll}
\displaystyle \min_{x} &\phi(r_1) + \dots + \phi(r_m)\\ 
\text{s.t. } & r = Ax-b
\end{array}
$$
where $\phi : \mathbb{R} \to \mathbb{R}$ is called the *penalty function*. We assume that $\phi$ is convex so that the problem is a convex optimisation problem. A penalty function assesses a cost or penalty for each component of the residual, i.e., $\phi(r_i)$. The total penalty is the sum of the penalties for each
residual. Different choices of $x$ lead to different resulting residuals and, therefore, different total penalties. While scaling the penalty function by a positive number does not affect the solution of the penalty function approximation problem (because this merely scales the objective function), the shape of the penalty function has a large effect on the solution of
the problem. If $\phi(r)$ is very small (or even zero) for small values of $r$, it means that we care very little (or not at all) if residuals have these values.
If $\phi(r)$ grows rapidly as $r$ becomes large, it means that we have a strong dislike for large residuals; if $\phi$ becomes infinite outside some interval, it means that residuals outside the interval are unacceptable.

Some common examples of penalty functions follow:

*   $\phi(u)=|u|^p$, where $p \ge 1$ corresponds to the $\ell_p$-norm approximation problem.
*   The deadzone-linear penalty function (with deadzone width $a > 0$) is given by
$$
\phi(u) = \left\{\begin{array}{ll}0 & |u|\le a \\|u|- a & |u|> a \end{array}\right.
$$
and gives no penalty for residuals smaller than $a$.
*   The log-barrier penalty function (with limit $a > 0$) has the form
$$
\phi(u) = \left\{\begin{array}{ll}-a^2 \log(1-(u/a)^2) & |u| < a \\\infty & |u| \ge a \end{array}\right.
$$
and gives an infinite penalty for residuals larger than $a$.

***Example 1.3:*** The figure below shows the hystogram of the residual amplitudes for four penalty functions. The sclaled penalty functions is shown for reference. The figure is produces by solving a problem with $n=30$ and $m=100$.

<div>
<img src="https://drive.google.com/uc?export=view&id=1H-HlC3e696pTmWsH3CRtVY_Kyv5tDYbH" width="600"/>
</div>

Figure 1.1. *Source: page 297 of $[1]$.*

One can note that

*   In comparison to the other penalty functions in the figure, the $\ell_1$-norm penalty puts the largest weight on small residuals and the least
weight on large residuals. Consequently, many residuals are either zero or very small. The optimal solution also has relatively more large residuals.
*   The $\ell_2$-norm penalty puts very small weight on small residuals, but strong weight on large residuals. The approximation has many modest residuals, and relatively few
larger ones.
*   The deadzone-linear penalty function puts no weight on residuals smaller than $0.5$ in absolute value, and relatively little weight on large residuals. We see that many residuals have the value $\pm 0.5$, right at the edge of the "free" zone, for which no penalty is assessed.
*   The log barrier penalty puts weight very much like the $\ell_2$-norm penalty for
small residuals, but puts very strong weight on residuals larger than around $0.8$, and infinite weight on residuals larger than $1$. We see that no residuals have a magnitude larger than 1, but otherwise the residual distribution is similar to the residual distribution for $\ell_2$-norm approximation.



**Sensitivity to outliers or large errors:** In the context of estimation, a very large residual would probably correspond to a faulty data or incorrect measurement. In this case it may be of interest to weigh less the outliers to avoid that the estimation is screwed towards the outliers. We have already seeen that the $\ell_1$-norm provides this feature by not weighing too strongly large residuals, which is why this estimator has been called *robust*. But sometime we may want a least-square behaviour for small values of the residuals. In this case, a penalty function that achieves *robust least-squares* is the *Huber penalty function* defined by
$$
\phi(u) = \left\{\begin{array}{ll}u^2 & |u|\le M \\M(2|u|- M) & |u|> M \end{array}\right.
$$
This function behaves like least-squares for values of the residuals less than the threshold $M$ and as a $\ell_1$-like linear function for larger residuals.

The effect of the Huber penalty function is shown in the figure below. 

<div>
<img src="https://drive.google.com/uc?export=view&id=1b6_5XDibyIIfZGPMBP3igPI8lI2cwNgg" width="400"/>
</div>

Figure 1.2. *Given 42 points represented by circles, the dashed line is the straight line approximator obtained using least-square. The presence of the two outliers screws the line. The solid line is the estimator obtained by using the Huber penalty function. Source: page 300 of $[1]$.* 





**Small residuals and sparsity:** From Figure 1.1 one can notice that the least-square penalty function puts very little weight on small residuals, while the dead-zone puts zero weight on small residuals. On the contrary, the $\ell_1$-norm penalty function (which we have already seen to be robust) produces many residuals which are very small or exactly zero. This is due to the fact that this norm continues to weigh small residuals in the same way as bigger residuals and there is always an incentive to push a residual to zero (while for least-square, for instance, the incentive decreases quadratically). Thus, the $\ell_1$-norm penalty function tends to produce *sparse* solutions which may be of interest for physical or numerical reasons.

**Approximation with constraints:** All the problems we saw above can be modified by adding convex constraints without any increase of computational complexity. The theoretical reasons for this will be explained later in the course but for now it suffices to know that constraints do not make our problem more complicated. Classical constraints are the following.

*   Nonnegativity constraints, i.e.
$$
\text{s.t. } x \succcurlyeq 0.
$$
This is useful if we know that our quantities have a physical meaning only if they are not negative, e.g., powers, intensities, rates.
*   Variable bounds, i.e.
$$
\text{s.t. } l \preccurlyeq x \preccurlyeq u.
$$
This is useful if we have prior knowledge that the variables lie in certain intervals.
*   Impose that the variables are probability distributions, i.e. nonnegative numbers that sum to 1
$$
\text{s.t. } x \succcurlyeq 0, \quad \mathbf{1}^{\top} x = 1.  
$$
This is useful when the estimated variables are proportions of a portfolio or normalized relative quantities.
*   Norm ball constraints of center $x_0$ and radius $d$, i.e.
$$
\text{s.t. } || x - x_0 || \le d  
$$
This is useful, for example, if we have a guess $x_0$ of what the variable can be and $d$ is the maximum deviation of the estimate from our guess which can be allowed. Or, in another example, the ball may represent a *trust region* in which $Ax$ is a good approximating model of an underlying more complex model $f(x)$. Outside the trust region the approximation $Ax$ is not a good representation of the model and the problem loses its meaning. This is for instance the case when considering the linear approximation $Ax$ computed around $x_0$ of the nonlinear model $f(x)$.


# 1.2 Least-Norm Problems

In [ ]:
from IPython.display import HTML
HTML('<iframe width="850" height="480" src="https://www.youtube.com/embed/4zYuLGCfOkY"></iframe>')

Consider a **control problem** in which we have $n$ unknown inputs variables $x_1, \dots, x_n$ that we want to determine. The $m$ equations $Ax=b$ represent $m$ specifications or requirements on the design. Among all the designs that satisfy the specifications, we may be interested in the design that uses the least amount of input "energy" as measured by a norm $|| \cdot ||$.  Mathematically the problem that we want to solve is

$$
\begin{array}{ll}
\displaystyle \min_{x} &||x||\\ 
\text{s.t. } & Ax=b
\end{array}
\tag{6}$$

where $A\in \mathbb{R}^{m\times n}$, $b \in \mathbb{R}^m$ and $x\in \mathbb{R}^n$. The problem $(6)$ is called **least-norm problem** and it is a simple example of constrained convex optimisation problem. We can assume without loss of generality that the rank of $A$ is full. When $m=n$ there is only one solution, i.e., $x=A^{-1}b$, in which case there is nothing to minimise. When $m > n$ there is either no feasible solution (because it is impossible to satisfy the constraints) or one solution (when $b$ is in the range of $A$). Thus, the least-norm problem is interesting only when $m < n$, i.e., there is an infinite number of solutions, and we want to select the one with minimum norm (note that $m < n$ is the opposite situation with respect to the norm approximation problem). 










Like for the norm approximation problem, Problem $(6)$ arises in a multitude of fields, beyond the control example used it above to introduce it.

**Reformulation as norm approximation problem:** Problem $(6)$ can be rewritten as Problem $(1)$. Let $x_0$ be any solution of $Ax=b$ and let $Z\in\mathbb{R}^{n \times k}$ be a matrix whose columns are a basis of the nullspace (or kernel) of $A$. Then the general solution $x$ of $Ax=b$ can be expressed as $x=x_0 + Zu$ for a generic $u\in\mathbb{R}^k$. This because $A(x_0 + Zu) = b + AZu = b + 0$. Then Problem $(6)$ becomes
$$ 
\displaystyle \min_u ||x_0 + Zu ||
$$
with variable $u\in\mathbb{R}^k$, which is a norm approximation problem.

**Estimation interpretation:** Let $x$ be a vector of parameters to be estimated and assume that we have $m < n$ noise-free linear measurements given by $Ax=b$. Since we have fewer measurements than parameters, $x$ is not fully determined. From prior knowledge we know that $x$ is more likely to be small than large (as measured with a certain norm). The least-norm
problem chooses the parameter vector $x$ that is smallest
(hence, most plausible) among all parameter vectors that are consistent with the
measurements $Ax = b$.

**Geometric interpretation:** Given a feasible set $\{x : Ax=b\}$, the objective is to find the distance between the set and $0$ (i.e. the closest $x$ in the set to $0$).

Some considerations are in order:

*   The most common least-norm problem involves the squared $\ell_2$ norm
$$
\begin{array}{ll}
\displaystyle \min_{x} &||x||_2^2\\ 
\text{s.t. } & Ax=b
\end{array}
$$
the unique solution of which is called least-squares solution of the equations $Ax=b$. This problem is one of the few problem that has an analytical solution. This can be found by writing the Lagrangian 
$$
L(x,\nu) = x^{\top}x + \nu^{\top}(Ax-b)
$$
and minimizing over $x$ and $\nu$, obtaining
$$
2x + A^{\top}\nu = 0 \qquad Ax-b=0
$$
which gives $x=-(1/2)A^{\top}\nu$. Replacing this in $Ax-b=0$ and solving for $\nu$ yields
$$
\nu^* = -2(AA^{\top})^{-1}b \qquad x^* = A^{\top}(AA^{\top})^{-1}b.
$$
Note that the Lagrangian was introduced in the Autumn Optimisation course but it will also be revised later in this course.
*   The least-norm problem can be formulated using general penalty functions and the same considerations described in the previous section will hold here as well. In particular, the $\ell_1$ norm tends to produce a sparse solution with a large number of zero components. Note that if the objective were to obtain exactly $m$ nonzero components, one would need to follow a different strategy by checking all combinations of $m \times m$ submatrices $A$ with the corresponding $m$ subvector $x$ and $b$. As long as the submatrix is nonsingular, there will be a unique solution. However, this means that all combinations of submatrices needs to be checked to find the one with the least-norm. Solving the least $\ell_1$-norm problem, on the other hand, gives a good heuristic for finding a sparse, and small, solution of $Ax = b$ (although, possibly, not with exactly $m$ nonzero components).

# 1.3 Regularized approximation

In [ ]:
from IPython.display import HTML
HTML('<iframe width="850" height="480" src="https://www.youtube.com/embed/bdXrG41zu3I"></iframe>')

Consider now the problem of finding a vector $x$ that is small and also make the residual $Ax-b$ small. This problem can be described with the **multi-objective optimisation problem**

$$
\displaystyle \min_x (||Ax-b||,||x||) \tag{7}
$$

where the two norms can be different in general. This problem can be seen as a variation of the least-square norm in which $Ax-b=0$ does not need to hold exactly. We are happy to trade some non-zero residual in exchange of a small $x$. Multi-obecjtive optimisation problems do not usually have a classical optimal solution. Most of the times there is a set of points which are not "comparable" with respect to one another (you cannot establish that one is better than the other). These optimal points represent a trade-off between the two objectives and can be found using several methods. The optimal trade-off curve of $||Ax-b||$ versus $||x||$, which shows how large one of the objectives must be made to have the other one small, can then be plotted. This curve is the curve of **Pareto optimal points**, which is a concept that generalizes optimality to multi-objective optimisation problems. For now, we postpone the formal definition of these points to a later chapter and we just try to solve the problem above based on the intuition that a Pareto optimal point is a point that cannot improve with respect to one obejective without making another objective worse.

**Regularization** is a common scalarization$^1$ method used to solve multi-objective problems. One form of regularization is to minimize the weighted sum of the objectives

$$
\min ||Ax-b|| + \gamma ||x|| \tag{8}
$$

where $\gamma >0$ is a problem parameter. As $\gamma$ varies from $0$ to $\infty$ the solution of $(8)$ traces out the optimal trade-off curve, i.e. the Pareto optimal points.

Regularisation is used in several contexts. In estimantion, the extra term penalising large $||x||$ can be interpreted as our prior knowledge that $||x||$ needs to be not too large. In optimal design, the term $||x||$ adds the cost of using large values of the design variables to the cost of missing the target specifications. The objective that $||x||$ be small can also reflect a modeling issue. It might be, for example, that the linear model $y = Ax$ is only a good approximation of the true nonlinear model $y = f(x)$ between $x$ and $y$ around 0. In order to have $f(x) \approx b$, we want $Ax \approx b$, and also
need $x$ small in order to ensure that $f(x) \approx Ax$. Regularization is also used when solving linear equations $Ax = b$ with $A$ poorly conditioned or even singular. Regularization gives a compromise between solving the equations (i.e., $Ax-b=0$) and keeping $x$ small.

**Footnote $^1$** A multi-objective optimisation problem is a special instance of **vector optimisation problem**, i.e. a problem in which the objective function is a vector. Thus, the word "scalarization" refers to the action of tranforming the vector objective into a scalar objective. For simplicity, we do not explain here the difference between multi-objective and vector optimisation problems.


**Tikhonov regularisation** (also called **Ridge Regression**) is a very common form of regularisation where the norms are Euclidean, namely

$$
\min ||Ax-b||^2_2 + \gamma ||x||^2_2.
$$

Note that this problem has an analytical solution (as usual with 2-norms, just take the derivative equal to zero)

$$
x = (A^\top A + \delta I)^{-1}A^\top b.
$$

Since $A^\top A + \delta I \succ 0$ for any $\delta>0$, we see how Tikhonov regularized problems require no rank condition on the matrix $A$.

More generally, one can weigh a function of $x$. For instance Tikhonov regularisation can be used to obtain a smooth solution by solving the problem

$$
\min ||Ax-b||^2_2 + \gamma ||\Delta x||^2_2 
$$

where $\Delta$ is a matrix such that $\Delta x$ is an discrete approximation of the second derivative of the parameters. This problem would promote solutions which have a small curvature. Yet more generally, multiple objectives can be considered, as in the next example.


**Example 1.4:** Consider a discrete-time dynamical system with scalar input sequence $u(0)$, $u(1)$, ..., $u(N)$ and scalar output sequence $y(0)$, $y(1)$, ..., $y(N)$ related by
$$
y(n) = \sum_{k=0}^n h(k) u(n-k)
$$
with $k$ and $n$ natural numbers. Our goal is to design the input sequence to achieve several goals.

*   We would like that the output $y$ tracks a desired reference signal $y_{\text{des}}$. This can be achieved by minimising the cost
$$
J_{\text{track}}= \frac{1}{N+1} \sum_{k=0}^N (y(k)-y_{\text{des}}(k))^2.
$$
*   Usually our control action costs us "energy", a quantity which is proportional to the square of the control inputs. To minime this cost, we use the objective
$$
J_{\text{mag}}= \frac{1}{N+1} \sum_{k=0}^N u(k)^2.
$$
*   In practice the actuators realizing the control sequence are physically limited and cannot in general apply wildly different inputs. Thus, we want to limit the variations of the input by minimizing the cost
$$
J_{\text{var}}= \frac{1}{N} \sum_{k=0}^{N-1} (u(k+1)-u(k))^2.
$$

In summary the problem that we want to minimize is the scalarized multiobjective optimisation problem

$$
\displaystyle \min_u J_{\text{track}} + \eta J_{\text{mag}} +\delta J_{\text{var}}.
$$

A simulation for different values of $\eta$ and $\delta$ is illustrated in the figure below.


<div>
<img src="https://drive.google.com/uc?export=view&id=1-cRWaajKuAjhLayOkpdiPc-w0dQr5Xwy" width="600"/>
</div>

Figure 1.3. *Optimal inputs (left) and resulting outputs (right) for three values of the regularization parameters $\delta$ and $\eta$. The top figures correspond to $\delta=0$ and small $\eta$. The tracking is good but the input is large and has large oscillations. The mid figures correspond to $\delta=0$ and larger $\eta$. The input is smaller but the tracking is worse. The variation of the input is still high. The bottom figures correspond to $\delta$ non zero and same $\eta$. The oscillations on the input disappeared, with no real worseing of the tracking performance. Source: page 309 of $[1]$.*

**Example 1.5:** Consider an audio signal $x_i$ sampled at time $i$. These samples are collected into a vector $x$. Unfortunately, our sampling introduces a noise, so the signal that is actually measured is 
$$
x_{\text{cor}} = x + v
$$
where $v$ is a small high frequency additive noise. Our objective is to estimate the original signal $x$ based on the corrupted signal $x_{\text{cor}}$. This process is called **signal reconstruction**, **de-noising** or **smoothing**. The problem that we want to solve if the multi-objective problem

$$
\displaystyle \min_{\hat x} (||\hat x-x_{\text{cor}}||_2,\phi(\hat x))
$$

where $\hat x$ is the variable to be estimated, $x_{\text{cor}}$ is a problem parameter and $\phi$ is a convex mapping which is called *regularisation function*, or smoothing objective. $\phi$ is meant to measure the roughness of the estimate $\hat x$. So the problem seeks an $\hat x$ which is close to $x_{\text{cor}}$ but that is also not rough. We will look at two selections for $\phi$: quadratic smoothing $\phi(x) = \sum_{i=1}^{n-1}( x_{i+1}- x_i)^2$, i.e. the $\ell_2$-norm of the differences, and total variation smoothing $\phi(x) = \sum_{i=1}^{n-1}| x_{i+1}- x_i|$, i.e. the $\ell_1$-norm of the differences. 

A simulation is illustrated in the next figure.



<div>
<img src="https://drive.google.com/uc?export=view&id=1h_qOA5Jm5lro4Q4X1js0TN361i0ROske" width="900"/>
</div>

Figure 1.4. *(Left) Top: original signal. Bottom: corrupted signal. (Right) Top three plots: reconstructed signal using quadratic smoothing from high (top) to low (bottom) $\gamma$ in the regularized problem. Bottom three plots: reconstructed signal using total variation smoothing from high (top) to low (bottom) $\gamma$ in the regularized problem. We observe that, unlike quadratic smoothing, total variation reconstruction preserves the sharp transitions in the signal. Source: page 315-317 of $[1]$.*

Finally, note that Tikhonov regularisation is not the only type of usuful regularisation. Another useful and common regularisation is the $\ell_1$ regularisation

$$
\min ||Ax-b||_2 + \gamma ||x||_1.
$$

As one can expect, this regularisation represents the trade-off between small residuals and sparse $x$.

# 1.4 Robust Approximation

The content of this section is advanced and you should not expect to be able to fully understand it at this stage. This topic is presented here to show you another important class of approximation problems. You should focus on understanding the practical importance of the problem, rather than the mathematical details. Later in the course, after the lecture on duality, you should come back to this section as you will then have the knowledge required to understand this topic.

In [ ]:
from IPython.display import HTML
HTML('<iframe width="850" height="480" src="https://www.youtube.com/embed/89SPTSqu8yE"></iframe>')

Consider again the basic approximation problem $||Ax - b||$ but now let us reflect on the fact that in most practical scenarios $A$ and/or $b$ are measured quantities. This means that most likely there is an error associated to each entry. For instance how can we be sure that our friction coefficient is really $0.586$ instead of $0.594$? This brings us to the problem of *robust approximation*, which tries to take into account that our model may be a non-exact representation of the true system. We see two approaches to deal with this problem: stochastic robust approximation and worst-case approximation.

**Stochastic Robust Approximation Problem:** In stochastic robust approximation we assume that $A$ is a random variable taking values in $\mathbb{R}^{m \times n}$ with mean $\bar A$, so that we can say that

$$
A = \bar A + U
$$

where $U$ is a random matrix with zero mean. Then we want to solve the problem

$$
\displaystyle \min_x \mathbf{E}\,||(\bar A + U)x -b||
$$

where $\mathbf{E}$ indicates the expectation operator. In general this problem is difficult to solve because it is difficult to evaluate the objective or its derivatives. There are however instances that can be solved. This is the case of the **stochastic robust least-square problem**

$$
\displaystyle \min_x \mathbf{E}\,||(\bar A + U)x -b||_2^2.
$$

In this case we have 

$$
\begin{array}{l}
\mathbf{E}\,||(\bar A + U)x -b||_2^2 = \mathbf{E}\,((\bar A + U)x -b)^\top (\bar A + U)x -b))\\
\qquad =(\bar A x - b)^\top(\bar A x - b) + \mathbf{E}\,x^\top U^\top U x = ||\bar Ax -b||_2^2 + x^\top P x
\end{array}
$$

with $P = \mathbf{E}\,U^\top U$, and where we have used the fact that if $z$ is not stochastic, then $\mathbf{E}z=z$. Thus, the stochastic robust least-square problem is nothing else than a Tikhonov regularised problem 

$$
\displaystyle \min_x ||\bar A x -b||_2^2 + ||P^{1/2}x||^2_2.
$$
with solution
$$
x = (\bar A^{\top} \bar A + P)^{-1} \bar A^\top b.
$$

**Worst-Case Robust Approximation Problem:** Another approach to robustness is to model the variations in the matrix $A$ using a set-based worst-case approach. In this case $A \in \mathcal{A} \in \mathbb{R}^{m \times n}$, where $\mathcal{A}$ is the set of all possible values for $A$. Then the worst-case robust approximation problem consists of minimising 

$$
\displaystyle \min_x \sup\{||Ax − b|| : A \in \mathcal{A}\}.
$$

Also in this case, this problem is in general difficult to solve. However, there are special cases that are tractable. One of this is the case in which the set $\mathcal{A}$ is described by $\mathcal{A} = \{ \bar A + u_1 A_1 + u_2 A_2 + \dots + u_p A_p : ||u|| \le 1 \}$, i.e. the image of a norm ball under an affine transformation. Then the worst-case error can be espressed as

$$
e_{wc}(x) = \sup_{||u|| \le 1} || (\bar A + u_1 A_1 + u_2 A_2 + \dots + u_p A_p)x -b || = \sup_{||u|| \le 1} ||P(x)u + q(x)||
$$

where $P(x) = \left[\begin{array}{cccc} A_1 x & A_2 x & \cdots A_p x   \end{array}\right]$ and $q(x) = \bar A x - b$. It turns out that for this formulation, both the worst-case Chebyshev approximation problem ($\ell_\infty$) and the worst-case least-square approximation problem ($\ell_2$)  are solvable. 





As an example, consider the worst-case least-square approximation


$$
\displaystyle \min_x e_{wc}(x) \tag{9}
$$

and note that $e_{wc}(x)$ can be written as the squareroot of the solution of the nonconvex quadratic optimisation problem

$$
\begin{array}{ll}
\max & ||P(x)u + q(x)||_2^2\\
s.t. & u^\top u \le 1. 
\end{array} \tag{10}
$$


Now, a solution to the worst-case least-square approximation problem $(9)$ is given by the minimum with respect to $x$ of the solution of the problem

$$
\begin{array}{ll}
\displaystyle\min_{t,\lambda} & t+\lambda\\
s.t. & \left[\begin{array}{ccc}I & P(x) & q(x)\\P(x)^\top & \lambda I & 0 \\ q(x)^\top & 0 & t\end{array}\right] \succcurlyeq 0. 
\end{array}\tag{11}
$$

At this point, you cannot know why this is the case. After the lecture on duality you are invited to come back to this example. It will then be clear that $(11)$ is the dual of $(10)$ and that $(11)$, being a semidefinite programme (SDP), can be solved by convex optimisation. Moreover, strong duality holds for this problem and so by minimising the solution of the dual $(11)$ with respect to $x$ one will find the solution of the original problem $(9)$. 




A typical comparison between nominal least-square, stochastic least-square and worst-case least-square is given in the figure below.

<div>
<img src="https://drive.google.com/uc?export=view&id=1SJZtdNk5xTQ11Dp8B3weEfL7pU3AF-7C" width="600"/>
</div>

Figure 1.5. *The nominal solution $x_{\text{nom}}$ achieves the best performance for $u=0$, which is the nominal problem, but it degrates quickly for large $u$. The stochastic solution $x_{\text{stoch}}$ does worse for the nominal solution but does better for large $u$. Finally, the worst-case $x_{\text{wc}}$ has a similar perfomance accross all $u$, quite bad for small $u$ but very good for large $u$. Source: page 320 of $[1]$.*

# 1.5 CVXPY

During each class we will formulate and solve at least one optimisation problem taken from an applied field. For the numerical solution of these problems you will use CVXPY.

CVXPY is an open source Python library for convex optimization problems. An advantage of CVXPY is that it allows you to express your problem in a natural way that follows the math.

As part of this chapter, read the following pages:

*   [What is CVXPY?](https://www.cvxpy.org/tutorial/intro/index.html)
*   [Disciplined Convex Programming](https://www.cvxpy.org/tutorial/dcp/index.html)
*   [Atomic Functions](https://www.cvxpy.org/tutorial/functions/index.html)

You should allocate around 60 minutes to this activity. You do not need to memorize anything, you will always have access to these pages during the class activities. However, if you don't read these pages in advance, you will not have any idea of where to start!

# End of CHAPTER 1
